# TODO

- Remove the food items from the lda as well
- Describe more what I do in this 
- Do LDA by poorly related restaurants

# Overview

In this notebook, I use LDA to model latent topics. Latent models don't always yield useful results and it is difficult to know whether they will be useful or not until we actually start. 

Steps:

1. Group the Data by restauarant and then squish the review text together. 

Extensions:

1. Consider only positive reviews. (Happy in the same say, upset for all different ways)
2. COnsider only negative reviews. 

In [1]:
import pandas as pd
from gensim.models.ldamulticore import LdaModel
import gensim
import nltk

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
combined_data = pd.read_hdf('../data/restaurant_reviews.hdf')

In [4]:
combined_data.columns

Index(['date', 'review_id', 'text', 'user_id', 'city', 'latitude', 'longitude',
       'name', 'neighborhoods', 'stars', 'hours'],
      dtype='object')

In [4]:
g_rest = combined_data.groupby(combined_data.name)

In [6]:
len(g_rest.stars.mean())

15321

In [7]:
text = g_rest.text.apply(lambda x: '\n'.join(x)).reset_index()

In [8]:
text.text.head()

0    So this place isn't spectacular, but it is def...
1    I really like this place. The portions are lar...
2    Delicious Sushi! Very big, fresh, tasty, flavo...
3    Pizza was just plain, nothing special. I have ...
4    I don't like the pizza hear and if you go for ...
Name: text, dtype: object

## Basic Preprocessing

In [9]:
text.text = text.text.apply(lambda x: gensim.utils.simple_preprocess(x))

In [10]:
text.text[:5]

0    [so, this, place, isn, spectacular, but, it, i...
1    [really, like, this, place, the, portions, are...
2    [delicious, sushi, very, big, fresh, tasty, fl...
3    [pizza, was, just, plain, nothing, special, ha...
4    [don, like, the, pizza, hear, and, if, you, go...
Name: text, dtype: object

In [9]:
# import spacy
# import os
# data_dir = '/Users/Will/anaconda/envs/py34/lib/python3.4/site-packages/spacy/en/data'
# nlp = spacy.en.English(data_dir=data_dir)

Now we need to make a gensim dictionary that 

In [11]:
dictionary = gensim.corpora.Dictionary(text.text)

In [12]:
len(dictionary)

242663

Now we get the Corpus into the format we want and save it to disk.

## Removing Stopwords

To start out with, we use the nltk stopwords. If this turns out to cause problems, we will also start excluding the most common words from the current corpus.

In [13]:
import nltk

In [14]:
stopwords = set(nltk.corpus.stopwords.words('english'))

In [15]:
stopwords_ids = list(map(dictionary.token2id.get, stopwords))

In [16]:
len(stopwords_ids)

127

In [17]:
dictionary.filter_tokens(stopwords_ids)

In [18]:
dictionary.filter_extremes(no_below=10, no_above=0.1)

In [19]:
len(dictionary)

41555

In [20]:
dictionary.compactify()

## Generating the Corpus

In [21]:
corpus = [dictionary.doc2bow(t) for t in text.text]

## Saving

In [23]:
gensim.corpora.MmCorpus.serialize('../data/deerwester.mm', corpus)

In [24]:
dictionary.save('../data/words.dict')

# Modelling

I should probably remove stopwords at some point

In [20]:
from gensim.corpora.dictionary import Dictionary
from gensim.corpora.mmcorpus import MmCorpus 
from gensim.models.ldamulticore import LdaMulticore

In [11]:
dictionary = Dictionary.load('../data/words.dict')

In [14]:
corpus = MmCorpus('../data/deerwester.mm')

I recommend running this on a power EC2 server. Or else it will take too long. 

In [ ]:
all_lda = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=20, workers=31,
                       chunksize=50, passes=1)

In [27]:
all_lda.save('../data/ldamodel')

In [33]:
print(all_lda)

LdaModel(num_terms=242663, num_topics=20, decay=0.5, chunksize=100)


In [35]:
all_lda.print_topics()

['0.050*the + 0.033*and + 0.027*to + 0.019*it + 0.016*of + 0.014*was + 0.014*is + 0.013*for + 0.013*in + 0.011*this',
 '0.054*the + 0.037*and + 0.023*to + 0.019*pizza + 0.019*it + 0.019*was + 0.016*of + 0.015*is + 0.013*for + 0.012*in',
 '0.056*the + 0.039*and + 0.023*to + 0.019*was + 0.018*it + 0.017*is + 0.016*of + 0.013*for + 0.011*in + 0.010*food',
 '0.056*the + 0.032*and + 0.024*to + 0.020*was + 0.019*of + 0.017*it + 0.014*for + 0.013*is + 0.012*in + 0.011*buffet',
 '0.053*the + 0.036*and + 0.026*to + 0.017*of + 0.016*it + 0.016*was + 0.014*is + 0.012*for + 0.011*in + 0.010*we',
 '0.055*the + 0.037*and + 0.022*was + 0.022*to + 0.019*it + 0.015*of + 0.013*for + 0.012*we + 0.011*is + 0.010*in',
 '0.057*the + 0.037*and + 0.022*to + 0.019*it + 0.018*was + 0.017*of + 0.015*is + 0.013*in + 0.011*for + 0.010*pho',
 '0.056*the + 0.035*and + 0.023*to + 0.018*it + 0.018*was + 0.016*burger + 0.014*of + 0.012*is + 0.012*in + 0.011*for',
 '0.053*the + 0.035*and + 0.022*to + 0.019*it + 0.019*wa

# Single Core

In [96]:
lda_single = LdaModel(corpus=corpus, id2word=dictionary, num_topics=20,
                     chunksize=300)

In [97]:
lda_single.print_topics()

['0.050*pad + 0.032*pittsburgh + 0.023*tom + 0.011*truck + 0.009*panang + 0.009*drunken + 0.007*ew + 0.006*satay + 0.006*papaya + 0.005*curries',
 '0.006*pepperoni + 0.004*subway + 0.004*zpizza + 0.004*philly + 0.003*factory + 0.003*refund + 0.003*pies + 0.002*hut + 0.002*cheesesteak + 0.002*wich',
 '0.021*brisket + 0.017*spaghetti + 0.012*ravioli + 0.010*lasagna + 0.009*ricotta + 0.009*bruschetta + 0.008*gelato + 0.008*marinara + 0.008*tiramisu + 0.007*pastas',
 '0.307*pho + 0.068*vietnamese + 0.033*unphogettable + 0.013*mi + 0.011*boba + 0.010*vermicelli + 0.010*sprouts + 0.010*brisket + 0.009*bo + 0.008*plaza',
 '0.013*charlotte + 0.006*grits + 0.004*cornbread + 0.004*tots + 0.003*vig + 0.003*uptown + 0.003*ahi + 0.003*soul + 0.003*kale + 0.002*lemonade',
 '0.034*pita + 0.031*hummus + 0.027*gyro + 0.018*falafel + 0.014*edinburgh + 0.013*feta + 0.012*mediterranean + 0.010*gyros + 0.007*kabob + 0.007*eastern',
 '0.019*pub + 0.013*madison + 0.010*shake + 0.008*pickles + 0.007*pretzel +

Saving it

In [98]:
lda_single.save('../data/all_lda')

# Positive Reviews Only

In [5]:
def generate_corpus(df, grouper='name', text='text', save=False):
    """
    Generates a dictionary with stopwords and most frequent words removed.
    Also tells how many items have been removed. Optionally allows saving the
    files to disk.


    :param df: pd.DataFrame()
    :param grouper: The column that we want to group by. For example, by restaurant.
    :param text: The column with the text that we want to model by
    :return: corpus, and dict object.

    Example
    ----
    >>> corpus, dictionary = generate_corpus(df, save=True) # doctest: +SKIP
    """
    # Combine the text on a restaurant level
    grouped_df = df.groupby(grouper)
    all_review_text = grouped_df[text].apply(lambda x: '\n'.join(x)).reset_index()

    # Do some Processing
    all_review_text[text] = all_review_text[text].apply(
        lambda x: gensim.utils.simple_preprocess(x))

    # Generate the dictionary
    dictionary = generate_dictionary(all_review_text[text])

    # Generate the corpus
    corpus = [dictionary.doc2bow(t) for t in all_review_text[text]]

    if save:
        try:
            gensim.corpora.MmCorpus.serialize('../data/corpus', corpus)
            dictionary.save('../data/words.dict')
        except IOError as e:
            logging.warning("Could not save to disk. Exception {}".format(e))

    return corpus, dictionary


def generate_dictionary(series):
    """
    Generates a gensim dictionary. 

    :param series:
    :return:
    """
    dictionary = gensim.corpora.Dictionary(series)
    original_dict_length = len(dictionary)

    stopwords = set(nltk.corpus.stopwords.words('english'))
    stopwords_ids = list(map(dictionary.token2id.get, stopwords))
    dictionary.filter_tokens(stopwords_ids)
    dictionary.filter_extremes(no_below=10, no_above=0.1)

    dictionary.compactify()
    percent_removed = 1 - (len(dictionary) / original_dict_length)
    logging.info('Removed {} items from our dictionary. {} %'
                 .format(original_dict_length - len(dictionary),
                         percent_removed))

    return dictionary

In [5]:
good_reviews = combined_data[combined_data.stars.isin([4,5])]

In [6]:
good_corpus, good_dict = generate_corpus(good_reviews)

In [14]:
from gensim import corpora
corpora.MmCorpus.serialize('../data/good_corpus.mm', good_corpus)
good_dict.save('../data/good_dict')

In [131]:
good_lda = LdaModel(corpus=good_corpus, id2word=good_dict, num_topics=20,
                     chunksize=300)

In [125]:
len(good_dict)

33514

In [134]:
good_lda.print_topics()

['0.033*pub + 0.026*deli + 0.016*pastrami + 0.015*bagel + 0.012*irish + 0.011*pretzel + 0.011*craft + 0.010*bagels + 0.010*subway + 0.009*ale',
 '0.063*pancakes + 0.037*waffle + 0.031*waffles + 0.031*benedict + 0.030*hash + 0.022*omelet + 0.017*pancake + 0.016*biscuits + 0.015*omelette + 0.014*syrup',
 '0.042*japanese + 0.031*sashimi + 0.023*ramen + 0.023*ayce + 0.022*tempura + 0.021*miso + 0.018*sake + 0.016*nigiri + 0.015*teriyaki + 0.013*yellowtail',
 '0.026*asada + 0.025*carne + 0.025*guacamole + 0.021*margaritas + 0.017*margarita + 0.015*chile + 0.015*tortillas + 0.014*enchiladas + 0.014*chipotle + 0.013*burritos',
 '0.135*pho + 0.067*indian + 0.044*vietnamese + 0.022*naan + 0.014*mi + 0.014*masala + 0.009*banh + 0.009*tikka + 0.008*yasu + 0.008*paneer',
 '0.037*le + 0.036*et + 0.033*montreal + 0.027*est + 0.019*les + 0.015*en + 0.014*une + 0.014*que + 0.014*des + 0.013*pas',
 '0.048*und + 0.031*der + 0.028*das + 0.027*ist + 0.022*sehr + 0.021*es + 0.019*ich + 0.018*auch + 0.017*a

In [136]:
good_lda.save('../data/good_lda')

## Negative Reviews Only

In [16]:
bad_reviews = combined_data[combined_data.stars.isin([1,2])]

In [17]:
bad_corpus, bad_dict = generate_corpus(bad_reviews)

In [132]:
bad_lda = LdaModel(corpus=bad_corpus, id2word=bad_dict, num_topics=20,
                   chunksize=300)

In [133]:
bad_lda.print_topics()

['0.005*airport + 0.004*pittsburgh + 0.003*flight + 0.003*fox + 0.002*presentation + 0.002*bay + 0.002*tuna + 0.002*wolfgang + 0.002*overrated + 0.002*puck',
 '0.023*chocolate + 0.022*dog + 0.015*toast + 0.015*cafe + 0.013*brunch + 0.013*waffle + 0.013*turkey + 0.011*dogs + 0.008*shop + 0.008*smoothie',
 '0.200*sushi + 0.023*tuna + 0.020*japanese + 0.016*sashimi + 0.016*tempura + 0.015*ayce + 0.012*teriyaki + 0.012*nigiri + 0.011*miso + 0.008*chefs',
 '0.017*burgers + 0.011*beers + 0.007*sports + 0.005*pub + 0.005*bun + 0.005*bartenders + 0.005*rings + 0.004*sliders + 0.004*tv + 0.004*nachos',
 '0.058*thai + 0.021*curry + 0.018*pad + 0.010*spaghetti + 0.009*basil + 0.009*tom + 0.008*olive + 0.008*meatballs + 0.006*garden + 0.006*lasagna',
 '0.041*lobster + 0.013*scallops + 0.011*filet + 0.010*lamb + 0.009*oysters + 0.009*risotto + 0.009*duck + 0.008*chocolate + 0.008*steaks + 0.007*calamari',
 '0.063*ramen + 0.056*korean + 0.031*tapas + 0.017*broth + 0.014*boba + 0.013*tofu + 0.010*bel

In [18]:
corpora.MmCorpus.serialize('../data/bad_corpus.mm', bad_corpus)
bad_dict.save('../data/bad_dict')

In [135]:
bad_lda.save('../data/bad_lda')

In [25]:
combined_data.head()

,date,review_id,text,user_id,city,latitude,longitude,name,neighborhoods,stars,hours
0,2012-12-01,6w6gMZ3iBLGcUM4RBIuifQ,This place was DELICIOUS!! My parents saw a r...,LWbYpcangjBMm4KPxZGOKg,Braddock,40.408735,-79.866351,Emil's Lounge,[],5,"{'Tuesday': {'close': '19:00', 'open': '10:00'..."
1,2013-03-15,jVVv_DA5mCDB6mediuwHAw,Can't miss stop for the best Fish Sandwich in ...,m1FpV3EAeggaAdfPx0hBRQ,Braddock,40.408735,-79.866351,Emil's Lounge,[],5,"{'Tuesday': {'close': '19:00', 'open': '10:00'..."
2,2013-03-30,3Es8GsjkssusYgeU6_ZVpQ,This place should have a lot more reviews - bu...,8fApIAMHn2MZJFUiCQto5Q,Braddock,40.408735,-79.866351,Emil's Lounge,[],5,"{'Tuesday': {'close': '19:00', 'open': '10:00'..."
3,2013-10-20,KAkcn7oQP1xX8KsZ-XmktA,This place was very good. I found out about Em...,uK8tzraOp4M5u3uYrqIBXg,Braddock,40.408735,-79.866351,Emil's Lounge,[],4,"{'Tuesday': {'close': '19:00', 'open': '10:00'..."
4,2013-11-07,BZNJkkP0bXnwQ2-sCqat2Q,"Old school.....traditional ""mom 'n pop"" qualit...",6wvlM5L4_EroGXbnb_92xQ,Braddock,40.408735,-79.866351,Emil's Lounge,[],5,"{'Tuesday': {'close': '19:00', 'open': '10:00'..."


# The Traits of Poorly Rated Restaurants

In [27]:
len(g_rest)

15321

In [6]:
avg_rest_ratings = g_rest.stars.mean()

In [7]:
len(avg_rest_ratings)

15321

In [20]:
well_reviewed = combined_data[combined_data.name.isin(well_reviewed.index)]

In [43]:
well_reviewed.index

Index(['#1 Brothers Pizza', '#1 Sushi', '1 Brothers Pizza',
       '10-to-10 In Delhi', '1000 Grammes', '108 Chinese Take Away',
       '11 Scalini', '131 Main', '188 Restaurant',
       '1923 Bourbon & Burlesque By Holly Madison',
       ...
       'noca', 'notion', 'parker's PGH', 's' Wölfle Bistro', 's.e.e.d. cafe',
       'the Goodwich', 'unPhogettable', 'z Fondue', 'Özge Imbiss',
       'é by José Andrés'],
      dtype='object', name='name', length=7385)

In [9]:
poorly_reviewed = avg_rest_ratings[avg_rest_ratings < 3.7]

In [45]:
poorly_reviewed.index

Index(['#1 Hawaiian Barbecue', '#1Brothers Pizza', '1 Brother's Pizza',
       '1 Eastern Super Buffet', '100% Natural Mexican Grill',
       '101 Asian Buffet', '101 Bistro', '1130 The Restaurant', '12 East Cafe',
       '12th Fairway Bar & Grill',
       ...
       'new siam', 'portale 50', 'reva zy', 'schuhs', 'sushi crystal',
       'täglich', 'zpizza', 'À Table', 'Ém Cafe', 'Ô Deux Soeurs'],
      dtype='object', name='name', length=7897)

In [10]:
poorly_reviewed_rests = combined_data[combined_data.name.isin(poorly_reviewed.index)]

In [11]:
len(poorly_reviewed_rests)

419117

In [12]:
bad_rest_corpus, bad_rest_dict = generate_corpus(poorly_reviewed_rests)

In [21]:
good_bus_corpus, good_bus_dict = generate_corpus(well_reviewed)

In [ ]:
df.groupby(['university', 'accepted']).trasnform(
    lambda x: x.fillna(x.mean()))

In [ ]:
data[condition, ]

# References

[Good tutorial](https://radimrehurek.com/gensim/tut1.html#corpus-formats) on how to get Corpora in the right format.  

http://nbviewer.ipython.org/gist/langmore/6820351